**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/trajectories.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=server_args)

# Let's do all of our imports here, too.
import meshcat
import numpy as np
import pydrake.all
import os
from IPython.display import display
from ipywidgets import Textarea
from functools import partial
import mcubes

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.examples.manipulation_station import ManipulationStation
import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets
from manipulation.utils import FindResource
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddWsg, AddShape
from manipulation.jupyter_widgets import MakePoseSlidersThatPublishOnCallback

In the cells below, I've told meshcat not to flush it's geometry everytime we run.  If you ever need to clear the geometry in the meshcat visualizer, just run this cell.

In [ ]:
meshcat.Visualizer(zmq_url=zmq_url).delete()

# Interactive inverse kinematics

This first cell gives us an interface that is very similar to the differential IK teleop interface that we used before.  See if you can spot any differences.

In [ ]:
def teleop_inverse_kinematics():
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa)
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(gripper_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(gripper_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
    MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WG)
    display(console)

teleop_inverse_kinematics()


This one has a collision to avoid.  Try moving it in negative y.

In [ ]:
meshcat.Visualizer(zmq_url=zmq_url).delete()

In [ ]:
def teleop_inverse_kinematics():
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa)
    box = AddShape(plant, pydrake.geometry.Box(0.1, 0.1, 1.0), "box")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("box", box), RigidTransform([0.25, 0.0, 0.5]))
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(gripper_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(gripper_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg))
    s = MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WG)
    s[4].value = -0.2
    s[5].value = 0.35  # set the initial z lower, to make the interaction interesting.
    display(console)

teleop_inverse_kinematics()


This one has the hand tracking a cylinder, but is allowed to touch anywhere along the cylinder.  The sliders are controlling the pose of the cylinder. Or you can set `grasp_cylinder` to `False` and just chase the robot around with a stick.

In [ ]:
meshcat.Visualizer(zmq_url=zmq_url).delete()

In [ ]:
def teleop_inverse_kinematics(grasp_cylinder=True):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa)
    cylinder = AddShape(plant, pydrake.geometry.Cylinder(0.02, 1.0), "cylinder")
    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    cylinder_body = plant.GetBodyByName("cylinder", cylinder)
    cylinder_frame = plant.GetFrameByName("cylinder", cylinder)

    console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

    def my_callback(context, pose):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(cylinder_frame, [0, 0, 0], plant.world_frame(), pose.translation(), pose.translation())
        ik.AddOrientationConstraint(cylinder_frame, RotationMatrix(), plant.world_frame(), pose.rotation(), 0.0)
        if grasp_cylinder:
            ik.AddPositionConstraint(gripper_frame, [0, 0.1, -0.02], cylinder_frame, [0, 0, -0.5], [0, 0, 0.5])
            ik.AddPositionConstraint(gripper_frame, [0, 0.1, 0.02], cylinder_frame, [0, 0, -0.5], [0, 0, 0.5])
        else:
            ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
        if not result.is_success():
            console.value += "IK failed\n"

    visualizer.load()
    X_WC = RigidTransform(RollPitchYaw(np.pi/2.0, 0, 0), [0.5, 0, 0.5])
    s = MakePoseSlidersThatPublishOnCallback(visualizer, context, my_callback, value=X_WC)
    display(console)

# Set grasp_cylinder=False if you just want to antagonize the robot with a stick.
teleop_inverse_kinematics(grasp_cylinder=True)


# Visualizing the configuration space

Note that I'm creating a second meshcat window for this visualization.

In [ ]:
meshcat.Visualizer(zmq_url=zmq_url).delete()

In [ ]:
def draw_configuration_space(shelves=True):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddPlanarIiwa(plant)
    wsg = AddWsg(plant, iiwa, roll=0.0)
    sphere = AddShape(plant, pydrake.geometry.Sphere(0.02), "sphere")
    X_WO = RigidTransform([0.6, 0, 0.65])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("sphere"), X_WO)

    if shelves:
        parser = pydrake.multibody.parsing.Parser(plant)
        bin = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", bin), RigidTransform([0.6,0,0.4]))

    plant.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    visualizer.load()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    sphere_frame = plant.GetFrameByName("sphere", sphere)

    ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant, plant_context)
    collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)
    grasp_constraint = ik.AddPositionConstraint(gripper_frame, [0, 0.1, 0], sphere_frame, [0, 0, 0], [0, 0, 0])
    
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.SetInitialGuess(q, q0)
    result = pydrake.solvers.mathematicalprogram.Solve(ik.prog())
    if not result.is_success():
        console.value += "IK failed\n"

    diagram.Publish(context)

    def eval(q0, q1, q2, c, tol):
        return float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

    v = meshcat.Visualizer(zmq_url=zmq_url2)
    v["/Background"].set_property("visible", False)
    v["initial_guess"].set_object(meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x66ffff))
    v["initial_guess"].set_transform(meshcat.transformations.translation_matrix(q0))
    v["ik_solution"].set_object(meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x6666ff))
    v["ik_solution"].set_transform(meshcat.transformations.translation_matrix(result.GetSolution(q)))

    low = plant.GetPositionLowerLimits()
    up = plant.GetPositionUpperLimits()
    N = 70 if running_as_notebook else 2
    vertices, triangles = mcubes.marching_cubes_func(tuple(low), tuple(up), N, N, N, 
                                                     partial(eval, c=grasp_constraint, tol=0.05), 0.5)
    v["grasp_constraint"].set_object(meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
                                     meshcat.geometry.MeshLambertMaterial(color=0x88ee88, wireframe=False))

    if shelves:
        vertices, triangles = mcubes.marching_cubes_func(tuple(low), tuple(up), N, N, N, 
                                                         partial(eval, c=collision_constraint, tol=0.0), 0.5)
        v["collision_constraint"].set_object(meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
                                        meshcat.geometry.MeshLambertMaterial(color=0xee8888, wireframe=False))


draw_configuration_space()